## Sprint アンサンブル学習

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。

ブレンディング

バギング

スタッキング

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.shape

(1460, 81)

In [4]:
X = df.loc[: , ["GrLivArea" , "YearBuilt"]]
y = df.loc[: , "SalePrice"]

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
              X , y , test_size = 0.2)

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import xgboost as xgb

lr = LinearRegression()
svm = SVR()
tree = DecisionTreeRegressor()
forest = RandomForestClassifier()
lgb = lgb.LGBMClassifier()
xgb = xgb.XGBRegressor(colsample_bytree = 0.5, learning_rate =0.1, max_depth = 2, n_estimators = 50,subsample =  1)

/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following comm

## 【問題1】ブレンディングのスクラッチ実装

ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。

精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

回帰では各モデルの平均値を最終的な出力とすることが一般的

In [7]:
def MSE(y_test , pred):
    return mean_squared_error(y_test , pred)

In [8]:
lr = LinearRegression()
svm = SVR()
tree = DecisionTreeRegressor()

lr.fit(X_train , y_train)
svm.fit(X_train , y_train)
tree.fit(X_train , y_train)
pred_lr = lr.predict(X_test)
pred_svm = svm.predict(X_test)
pred_tree = tree.predict(X_test)


mse_lr = MSE(y_test , pred_lr)
mse_svm = MSE(y_test , pred_svm)
mse_tree = MSE(y_test , pred_tree)
# mse_mean = (mse_lr+mse_svm+mse_tree )/3

In [9]:
print("線形回帰のSME : {:.2f}".format(mse_lr))
print("SVMのSME : {:.2f}".format(mse_svm))
print("決定木のSME : {:.2f}".format(mse_tree))
# print("平均SME : {:.2f}".format(mse_mean))

線形回帰のSME : 1381777484.71
SVMのSME : 4624900550.36
決定木のSME : 2281769631.37


In [10]:
# models = [lr, svm, tree]
# model_names = ["Linear Regression", "SVM", "Decision Tree"]
# def score(X_train , X_test , y_train , y_test):
#     for model , name in zip(models, model_names):
#         model.fit(X_train,y_train)
#         print(name , model.score(X_test , y_test))

・SVMのスコアが異常に悪い

・前処理やハイパーパラメーターを変えて検証していく

## 重み（線形回帰、SVM、決定木）

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
              X , y , test_size = 0.2)

lr.fit(X_train , y_train)
svm.fit(X_train , y_train)
tree.fit(X_train , y_train)
pred_lr = lr.predict(X_test)
pred_svm = svm.predict(X_test)
pred_tree = tree.predict(X_test)

pred_sum = pred_lr*0.5 + pred_svm*0.1 + pred_tree*0.4

mse_lr = MSE(y_test , pred_lr)
mse_svm = MSE(y_test , pred_svm)
mse_tree = MSE(y_test , pred_tree)
mse_mean = (mse_lr+mse_svm+mse_tree )/3

In [12]:
print("線形回帰のSME : {:.2f}".format(mse_lr))
print("SVMのSME : {:.2f}".format(mse_svm))
print("決定木のSME : {:.2f}".format(mse_tree))
print("平均SME : {:.2f}".format(mse_mean))

線形回帰のSME : 3164596808.71
SVMのSME : 10190731994.87
決定木のSME : 4745151314.36
平均SME : 6033493372.65


重みを変えSMEが減少
平均のSMEも減少した

## 標準化（線形回帰、SVM、決定木）

In [13]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_std = std.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
              X_std , y , test_size = 0.2)

lr.fit(X_train , y_train)
svm.fit(X_train , y_train)
tree.fit(X_train , y_train)
pred_lr = lr.predict(X_test)
pred_svm = svm.predict(X_test)
pred_tree = tree.predict(X_test)

pred_sum = pred_lr + pred_svm + pred_tree

mse_lr = MSE(y_test , pred_lr)
mse_svm = MSE(y_test , pred_svm)
mse_tree = MSE(y_test , pred_tree)
mse_mean = (mse_lr+mse_svm+mse_tree )/3

In [14]:
print("線形回帰のSME : {:.2f}".format(mse_lr))
print("SVMのSME : {:.2f}".format(mse_svm))
print("決定木のSME : {:.2f}".format(mse_tree))
print("平均SME : {:.2f}".format(mse_mean))

線形回帰のSME : 1892798287.93
SVMのSME : 7549005683.31
決定木のSME : 2278002319.77
平均SME : 3906602097.00


標準化によってもSMEが減少した

## モデルを入れ替える（線形回帰、lgbm、決定木）

SVMのスコアが悪いため、SVMとlgbmを入れ替えて実行する

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
              X , y , test_size = 0.2)

lr.fit(X_train , y_train)
lgb.fit(X_train , y_train)
tree.fit(X_train , y_train)
pred_lr = lr.predict(X_test)
pred_lgb = lgb.predict(X_test)
pred_tree = tree.predict(X_test)

pred_sum = pred_lr + pred_lgb + pred_tree

mse_lr = MSE(y_test , pred_lr)
mse_lgb = MSE(y_test , pred_lgb)
mse_tree = MSE(y_test , pred_tree)
mse_mean = (mse_lr+mse_lgb+mse_tree )/3

/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
print("線形回帰のSME : {:.2f}".format(mse_lr))
print("lgbのSME : {:.2f}".format(mse_lgb))
print("決定木のSME : {:.2f}".format(mse_tree))
print("平均SME : {:.2f}".format(mse_mean))

線形回帰のSME : 1967803547.34
lgbのSME : 3204050894.29
決定木のSME : 3140644437.73
平均SME : 2770832959.79


スコアが大幅に向上した
平均値も一番優れていた

試したこと
・重みの初期値の変更
・標準化
・モデルの変更

## 【問題2】バギングのスクラッチ実装

バギングは入力データの選び方を多様化する方法です。

学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。

それらによってモデルをN個学習し、推定結果の平均をとります。

ブレンディングと異なり、それぞれの重み付けを変えることはありません。

In [79]:
df = pd.read_csv("train.csv")
X = df.loc[: , ["GrLivArea" , "YearBuilt"]]
y = df.loc[: , "SalePrice"]
# std = StandardScaler()
# X_std = std.fit_transform(X)
# X_std = pd.DataFrame(data = X_std)
X_train , X_test ,y_train ,  y_test = train_test_split(
             X , y , test_size = 0.2)

In [80]:
X_t = np.random.choice(X_train.index , X_train.shape[0] , replace=True)#.tolist()

In [81]:
X_t

array([ 962,  797,  395, ..., 1329,   67,  809])

In [82]:
X_train.shape

(1168, 2)

In [83]:
X_train.shape

(1168, 2)

In [84]:
X_train.loc[X_t].shape

(1168, 2)

In [143]:
sme_list = []
def bagging(model , X , y , n = 10):
    for i in range(n):
        X_train , X_test ,y_train ,  y_test = train_test_split(
             X , y , test_size = 0.2)
        X_index = np.random.choice(X_train.index , X_train.shape[0] , replace=True)
        X_train_n = X_train.loc[X_index]
        model.fit(X_train_n , y_train)
        pred = model.predict(X_test)
        sme = mean_squared_error(y_test , pred)
        sme_list.append(sme)
#     print("スコア{}".format(model.score(X_test , y_test)))
    return (np.mean(sme_list[-1]))

In [144]:
bagging(lr , X , y) 

4928511350.220617

In [63]:
4523739118.106428 - 7739357724.748134

-3215618606.6417055

線形回帰を用いて良い精度が確認できた

## 【問題3】スタッキングのスクラッチ実装

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
K0 = 3
,
M0 = 2
 程度にします。

《学習時》

（ステージ 
0
 ）

学習データを 
K
0
 個に分割する。
分割した内の 
(
K
0
−
1
)
 個をまとめて学習用データ、残り 
1
 個を推定用データとする組み合わせが 
K
0
 個作れる。
あるモデルのインスタンスを 
K
0
 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り 
1
 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも 
K
0
 個用意し、同様のことを行う。モデルが 
M
0
 個あれば、 
M
0
 個のブレンドデータが得られる。
（ステージ 
n
 ）

ステージ 
n
−
1
 のブレンドデータを
M
n
−
1
 次元の特徴量を持つ学習用データと考え、 
K
n
 個に分割する。以下同様である。
（ステージ 
N
 ）＊最後のステージ

ステージ 
N
−
1
 の 
M
N
−
1
 個のブレンドデータを
M
N
−
1
 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。
《推定時》

（ステージ 
0
 ）

テストデータを 
K
0
×
M
0
 個の学習済みモデルに入力し、
K
0
×
M
0
 個の推定値を得る。これを 
K
0
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）
（ステージ 
n
 ）

ステージ 
n
−
1
 で得たブレンドテストを 
K
n
×
M
n
 個の学習済みモデルに入力し、
K
n
×
M
n
 個の推定値を得る。これを 
K
n
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）
（ステージ 
N
 ）＊最後のステージ

ステージ 
N
−
1
 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [26]:
#クロスバリデーション
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

## 読み込みに時間がかかるためGridSearchCVを実行するコードは必要時以外はコメントアウトにしておく

In [27]:
from sklearn.model_selection import GridSearchCV

# parameters = {"n_estimators" : [50],
#          "learning_rate":[0.1,0.3,0.5],
#         "max_depth": [2,3,5,10],
#          "subsample":[0.5,0.8,0.9,1],
#          "colsample_bytree": [0.5,1.0],
#          }

# grid = GridSearchCV(estimator = xgb , param_grid = parameters , cv = kf , scoring = "mean_squared_error")
# grid.fit(preds , y_train)
# print(grid.best_estimator_)

In [28]:
# parameters = { "n_estimators":[i for i in range(10,50,10)],
#     "criterion":["gini","entropy"],
#     "max_depth":[i for i in range(1,5,1)],
#      'min_samples_split': [2, 4, 10,12,16],
#     "random_state":[3],
#          }

# grid = GridSearchCV(estimator = forest , param_grid = parameters , cv = kf , scoring = "mean_squared_error")
# grid.fit(preds , y_train)
# print(grid.best_estimator_)

In [29]:
# print(grid.best_score_)
# print(grid.best_params_)

In [30]:
df = pd.read_csv("train.csv")
X = df.loc[: , ["GrLivArea" , "YearBuilt"]]
y = df.loc[: , "SalePrice"]
X_train , X_test ,y_train ,  y_test = train_test_split(
             X , y , test_size = 0.2)

In [31]:
#クロスバリデーション
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [32]:
# xgb = xgb.XGBRegressor(colsample_bytree = 0.5, learning_rate =0.1, max_depth = 2, n_estimators = 50,subsample =  1)
forest = RandomForestClassifier(max_depth = 2, min_samples_split = 2, n_estimators = 40, random_state =3)

In [205]:
# Stacking(models , X_train ,X_test,y_train)
X_std = std.fit_transform(X)
X_std = pd.DataFrame(data = X_std)
X_train , X_test ,y_train ,  y_test = train_test_split(
             X_std , y , test_size = 0.2)
#クロスバリデーション
from sklearn.model_selection import KFold
kf = KFold(n_splits = 3)

In [213]:
models = [xgb , lgb , lr]
def Stacking(models , X_train , X_test , y_train):
    preds = np.array([])
    preds_test = []
    va_idxes = []
    kf = KFold(n_splits = 3 , random_state = 5)

    #クロスバリデーションで学習、予測を行い、予測値とインデックスを保存
    for m, model in enumerate(models):
        for i , (tr_idx , va_idx) in enumerate(kf.split(X_train)):
            tr_X , va_X = X_train.iloc[tr_idx] , X_train.iloc[va_idx]
            tr_y , va_y = y_train.iloc[tr_idx] , y_train.iloc[va_idx]
            model.fit(tr_X , tr_y) #va_X , va_y)
            model.fit(va_X , va_y)
            pred = model.predict(va_X)
            preds = np.append(preds , pred)
            pred_test = model.predict(X_test)
            preds_test = np.append(preds_test , pred_test)
            va_idxes.append(va_idx)
    #バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    preds = preds.reshape(1168 , len(models) )
    preds_test = preds_test.reshape(292 , 9)
    va_idxes = np.concatenate(va_idxes)
    #preds = np.concatenate(preds , axis = 0)
    order = np.argsort(va_idxes)
#    pred_train = preds[order]
    
    #テストデータに対する予測値の平均をとる
    preds_test_1 = np.mean(preds_test[: , :3] , axis = 1).reshape(-1,1)
    preds_test_2 = np.mean(preds_test[: , 3:6] , axis = 1).reshape(-1,1)
    preds_test_3 = np.mean(preds_test[: , 6:] , axis = 1).reshape(-1,1)
    preds_test_sum = np.concatenate([preds_test_1 , preds_test_2 , preds_test_3] , axis=1)
#     pred_train = pred_train.reshape(1168  , len(models))
    return preds_test_sum , preds

In [214]:
preds_test , preds =Stacking(models , X_train ,X_test,y_train)

/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/py

[02:35:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:35:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/hayashikentaro/.pyenv/versions/anaconda3-5.3.

In [215]:
X_train.shape

(1168, 2)

In [216]:
preds.shape

(1168, 3)

In [217]:
preds_test.shape

(292, 3)

In [218]:
def model(X_train , X_test , y_train , model):
    model.fit(preds , y_train)
    pred = model.predict(preds_test)
    sme = mean_squared_error(y_test , pred)
    return sme

In [219]:
model(preds , preds_test , y_train , lr)

5841546286.956702

モデル３、K-Fold3にて実施
ステージ０にて線形回帰、XGB、lgb似て学習
ステージNにて再度線形回帰を実施

今回のスクラッチを通してアンサンブル学習という技法を知り、引き出しが増えたと感じたがバリエーションの幅が広すぎて扱うのに苦労しそうな印象を感じた（パラメータチューニングなども入れると無限に組み合わせがある）
しかし今回はグリッドサーチの復習にもなり機械学習の面白さの根源を知れてよかったと個人的に感じた